# Bronze & Silver Export Inspection

Interactively explore the bronze (`fact_long`) and silver (`statement_lines`, `statement_facts`) outputs generated by `export_tables.py`.

The next cell auto-detects the most recent exported filing; update `CIK`/`ACCESSION` there if you want to override.


In [3]:
from pathlib import Path
import json
import pandas as pd

def _detect_base_dir() -> Path:
    search_roots = [Path.cwd()] + list(Path.cwd().parents)
    for root in search_roots:
        candidate = root / 'data' / 'bronze'
        if candidate.exists():
            return root / 'data'
    raise FileNotFoundError("Could not find a 'data/bronze' directory. Run export_tables.py first or set BASE_DIR manually.")

try:
    BASE_DIR
except NameError:
    BASE_DIR = _detect_base_dir()
else:
    BASE_DIR = Path(BASE_DIR)
    if not (BASE_DIR / 'bronze').exists():
        BASE_DIR = _detect_base_dir()

BASE_DIR = Path(BASE_DIR)
bronze_root = BASE_DIR / 'bronze'

available = sorted(
    [
        (path.parent.name, path.name)
        for path in bronze_root.glob('*/*')
        if (path / 'metadata.json').exists()
    ]
)

if not available:
    raise FileNotFoundError("No bronze exports found; run export_tables.py first.")

if 'CIK' in globals() and 'ACCESSION' in globals():
    candidate = bronze_root / str(CIK) / str(ACCESSION)
    if not (candidate / 'metadata.json').exists():
        print(f"Specified CIK/accession not found; falling back to {available[-1]}")
        CIK, ACCESSION = available[-1]
else:
    CIK, ACCESSION = available[-1]

bronze_path = bronze_root / CIK / ACCESSION
silver_path = BASE_DIR / 'silver' / CIK / ACCESSION

bronze_path, silver_path


(PosixPath('/Users/yanyan/Agent/SECDataExtractor_v3/data/bronze/0001318605/0001628280-25-003063'),
 PosixPath('/Users/yanyan/Agent/SECDataExtractor_v3/data/silver/0001318605/0001628280-25-003063'))

In [4]:
# Load filing metadata extracted during export
with (bronze_path / "metadata.json").open() as handle:
    metadata = json.load(handle)
metadata

{'filing_id': '0001318605-000162828025003063',
 'entity_cik': '0001318605',
 'accession_number': '0001628280-25-003063',
 'company_name': 'Tesla, Inc.',
 'form_type': '10-K',
 'filing_date': '2025-01-30T00:00:00'}

In [5]:
# Load bronze fact_long table
fact_long = pd.read_parquet(bronze_path / "fact_long.parquet")
fact_long.head()

,filing_id,entity_cik,accession_number,statement_role_id,statement_role_uri,role_name,statement_short_name,statement_type,metalink_group_type,metalink_subgroup_type,...,unit,entity,dimension_json,dimension_hash,is_consolidated,fact_id,is_extension,concept_balance,concept_datatype,has_negated_label
0,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,iso4217:USD,e:0001318605,None,None,True,f-54,False,None,None,False
1,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,iso4217:USD,e:0001318605,None,None,True,f-55,False,None,None,False
2,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,iso4217:USD,e:0001318605,None,None,True,f-56,False,None,None,False
3,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,iso4217:USD,e:0001318605,None,None,True,f-57,False,None,None,False
4,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,iso4217:USD,e:0001318605,None,None,True,f-58,False,None,None,False


In [6]:
# Inspect schema & row counts
{
    "columns": fact_long.columns.tolist(),
    "row_count": len(fact_long)
}

{'columns': ['filing_id',
  'entity_cik',
  'accession_number',
  'statement_role_id',
  'statement_role_uri',
  'role_name',
  'statement_short_name',
  'statement_type',
  'metalink_group_type',
  'metalink_subgroup_type',
  'line_order',
  'concept_qname',
  'label',
  'label_terse',
  'label_standard',
  'period_label',
  'period_end',
  'period_start',
  'is_instant',
  'value_raw',
  'value_numeric',
  'decimals_hint',
  'unit',
  'entity',
  'dimension_json',
  'dimension_hash',
  'is_consolidated',
  'fact_id',
  'is_extension',
  'concept_balance',
  'concept_datatype',
  'has_negated_label'],
 'row_count': 425}

In [7]:
# Load silver statement tables
statement_lines = pd.read_parquet(silver_path / "statement_lines.parquet")
statement_facts = pd.read_parquet(silver_path / "statement_facts.parquet")
statement_lines.head()

,filing_id,entity_cik,accession_number,statement_role_id,statement_role_uri,role_name,statement_short_name,statement_type,metalink_group_type,metalink_subgroup_type,line_order,concept_qname,label,label_terse,label_standard,is_abstract,preferred_label_role,line_depth,is_extension
0,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,1,us-gaap:StatementOfFinancialPositionAbstract,Statement of Financial Position [Abstract],None,None,True,None,0,False
1,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,2,us-gaap:AssetsAbstract,Assets,None,None,True,None,1,False
2,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,3,us-gaap:AssetsCurrentAbstract,Current assets,None,None,True,None,2,False
3,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,4,us-gaap:CashAndCashEquivalentsAtCarryingValue,Cash and cash equivalents,None,None,False,None,3,False
4,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,5,us-gaap:ShortTermInvestments,Short-term investments,None,None,False,None,3,False


In [8]:
statement_facts.head()

,filing_id,entity_cik,accession_number,statement_role_id,statement_role_uri,role_name,statement_short_name,statement_type,metalink_group_type,metalink_subgroup_type,...,value_raw,value_numeric,decimals_hint,unit,entity,dimension_json,dimension_hash,is_consolidated,fact_id,is_extension
0,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,16139000000,1.613900e+10,-6.0,iso4217:USD,e:0001318605,None,None,True,f-54,False
1,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,16398000000,1.639800e+10,-6.0,iso4217:USD,e:0001318605,None,None,True,f-55,False
2,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,20424000000,2.042400e+10,-6.0,iso4217:USD,e:0001318605,None,None,True,f-56,False
3,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,12696000000,1.269600e+10,-6.0,iso4217:USD,e:0001318605,None,None,True,f-57,False
4,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,4418000000,4.418000e+09,-6.0,iso4217:USD,e:0001318605,None,None,True,f-58,False


In [9]:
# Quick rollups by statement
statement_lines.groupby("statement_short_name").size().to_frame("line_rows")

,line_rows
statement_short_name,
Balance Sheet,51
Cash Flows,47
Equity,40
Income Statement,49


In [10]:
# Example: Focus on Balance Sheet facts for the latest period
balance_sheet_facts = (
    statement_facts
    .query("statement_short_name == 'Balance Sheet'")
    .sort_values(["concept_qname", "period_end"])
)
balance_sheet_facts.head(20)

,filing_id,entity_cik,accession_number,statement_role_id,statement_role_uri,role_name,statement_short_name,statement_type,metalink_group_type,metalink_subgroup_type,...,value_raw,value_numeric,decimals_hint,unit,entity,dimension_json,dimension_hash,is_consolidated,fact_id,is_extension
35,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,9080000000,9.080000e+09,-6.0,iso4217:USD,e:0001318605,None,None,True,f-89,True
34,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,10723000000,1.072300e+10,-6.0,iso4217:USD,e:0001318605,None,None,True,f-88,True
21,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,184000000,1.840000e+08,-6.0,iso4217:USD,e:0001318605,None,None,True,f-75,True
20,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,1076000000,1.076000e+09,-6.0,iso4217:USD,e:0001318605,None,None,True,f-74,True
15,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,5229000000,5.229000e+09,-6.0,iso4217:USD,e:0001318605,"{""us-gaap:PropertyPlantAndEquipmentByTypeAxis""...",997105e9f560adb4d6d59e7e64d812b00e6ad52f,False,f-69,True
14,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,4924000000,4.924000e+09,-6.0,iso4217:USD,e:0001318605,"{""us-gaap:PropertyPlantAndEquipmentByTypeAxis""...",997105e9f560adb4d6d59e7e64d812b00e6ad52f,False,f-68,True
39,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,2373000000,2.373000e+09,-6.0,iso4217:USD,e:0001318605,None,None,True,f-93,True
38,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,2456000000,2.456000e+09,-6.0,iso4217:USD,e:0001318605,None,None,True,f-92,True
43,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,2857000000,2.857000e+09,-6.0,iso4217:USD,e:0001318605,None,None,True,f-97,True
42,0001318605-000162828025003063,0001318605,0001628280-25-003063,ns14,,9952151 - Statement - Consolidated Balance Sheets,Balance Sheet,balance_sheet,statement,,...,5757000000,5.757000e+09,-6.0,iso4217:USD,e:0001318605,None,None,True,f-96,True
